In [1]:
# Preprocessing
from pathlib import Path
import numpy as np
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler, FunctionTransformer, PolynomialFeatures
from sklearn.model_selection import train_test_split, TimeSeriesSplit, GridSearchCV
# from workalendar.europe import France

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from catboost import CatBoostRegressor, Pool
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from timeit import default_timer as timer

plt.style.use('fivethirtyeight')

path = Path("mdsb-2023")

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [9]:
def train_test_split_temporal(X, y, delta_threshold="60 days"):
    
    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

In [2]:
def add_lags(X, cols_to_lag=['t', 'u', 'vv', 'nnuage4'], lag_list=[2, -24, -2]):
    X = X.copy()

    feature_columns = [col for col in X.columns if col in cols_to_lag]

    for l in lag_list:
        lag_columns = [f'{col}_lag{l}' for col in feature_columns]
        X[lag_columns] = X[feature_columns].shift(periods=l, axis=0)
        X[lag_columns] = X[lag_columns].interpolate(method='linear').interpolate(method='bfill').interpolate(method='ffill')

    return X

def add_moving_average(X, cols_to_ma=['t', 'u', 'vv', 'nnuage4'], window_list=[24*7, 24], centered=True):
    X = X.copy()

    feature_columns = [col for col in X.columns if col  in cols_to_ma]

    for w in window_list:
        ma_columns = [f'{col}_ma{w}' for col in feature_columns]
        X[ma_columns] = X[feature_columns].rolling(window=w, center=centered).mean()
        X[ma_columns] = X[ma_columns].interpolate(method='linear').interpolate(method='bfill').interpolate(method='ffill')

    return X

### Define pipeline functions

In [3]:
def _encode_dates(X, col_name='date'):
    X = X.copy()

#     X["year"] = X[col_name].dt.year
    X["month"] = X[col_name].dt.month
#     X["day"] = X[col_name].dt.day
    X["weekday"] = X[col_name].dt.weekday
    X["hour"] = X[col_name].dt.hour
#     X['isweekend'] = X[col_name].apply(lambda x: 0 if x.weekday() in range(5) else 1)

#     cal = France()
#     X['is_holiday'] = X['date'].apply(cal.is_holiday)
#     X['working_day'] = X['date'].apply(cal.is_working_day)


    X[["month", "weekday", "hour"]] = X[["month", "weekday", "hour"]].astype("category")

    return X.drop(columns=[col_name])

In [4]:
def _encode_covid(X, col_name='date'):
    X = X.copy()
    
    # Create masks for lockdown dates
    lockdown_1 = ((X['date'] >= '2020-10-17')
                      & (X['date'] <= '2020-12-14') )#& ((X['date'].dt.hour >= 21) | (X['date'].dt.hour <= 6))

    lockdown_2 = ((X['date'] >= '2020-12-15')
                      & (X['date'] <= '2021-02-26') )#& ((X['date'].dt.hour >= 18) | (X['date'].dt.hour <= 6))

    lockdown_3 = ((X['date'] >= '2021-02-27')
                      & (X['date'] <= '2021-05-02') )#& ((X['date'].dt.hour >= 19) | (X['date'].dt.hour <= 6))

    X['Covid'] = 0
    X.loc[lockdown_1 | lockdown_2 | lockdown_3, 'Covid'] = 1

    return X

In [5]:
def _merge_external_data(X, include_lags=True, include_ma=True):
    to_keep = ['date', 'hnuage4', 't', 'ctype4', 'nnuage4', 'u', 'etat_sol', 'perssfrai',
               'tx12', 'cm', 'tn12', 'tend24', 'vv', 'rafper', 'rr24', 'hnuage2', 'td', 'rr3', 'hnuage3', 'hnuage1']
    
    ext_data = pd.read_csv(path / 'external_data.csv', parse_dates=['date'])[to_keep]
    
    ext_data.drop(columns=ext_data.columns[ext_data.isna().sum()>1000])
    
    full_date_range = pd.date_range(start=np.min([np.min(data.date), np.min(test.date)]), end=np.max([np.max(data.date), np.max(test.date)]), freq='H')
    full_date_range = pd.DataFrame({'date': full_date_range})

    ext_data = full_date_range.merge(ext_data, on='date', how='left')
    
    columns_to_interpolate = ext_data.drop(columns='date').columns
    ext_data[columns_to_interpolate] =  ext_data[columns_to_interpolate].interpolate(method='polynomial', order=3).interpolate(method='bfill').interpolate(method='ffill')
    
    if include_lags:
        ext_data = add_lags(ext_data)
    
    if include_ma:
        ext_data = add_moving_average(ext_data)
    
    to_drop = [
        'vv_lag2', 't_lag-24', 'u_lag-24', 'vv_lag-24', 'tx12', 'nnuage4_lag-2', 'etat_sol',
        'vv_lag-2', 'u', 'nnuage4', 'ctype4', 't', 'u_lag2', 'vv_ma24', 't_lag2', 'tend24',
        'u_ma24', 'nnuage4_ma24', 'vv_ma168', 'cm', 'hnuage1', 'hnuage3', 'rr3', 'td',
        'hnuage2', 'rr24', 'rafper', 'vv', 'u_ma168', 'hnuage4'
    ]

    ext_data.drop(columns=to_drop, inplace=True)

    X = X.copy()

    X["date"] = X["date"].astype('datetime64[ns]')
    ext_data["date"] = ext_data["date"].astype('datetime64[ns]')

    X["orig_index"] = np.arange(X.shape[0])
    
    X = pd.merge_asof(X.sort_values("date"), ext_data.sort_values("date"), on="date")

    # Sort back to the original order
    X = X.sort_values("orig_index")
    del X["orig_index"]

    return X

In [6]:
def _gas_price_encoder(X):
    X = X.copy()
    X['gas_price'] = 1
    
    gas_prices = np.array([
        1.22, 1.21, 1.22, 1.27,
        1.31, 1.36, 1.4, 1.39, 1.4, 1.43, 1.45, 1.45, 1.46, 1.56
    ])
    
    years = [
        2020, 2020, 2020, 2020,
        2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021, 2021
    ]
    
    months = [
        9, 10, 11, 12,
        1, 2, 3, 4, 5, 6, 7, 8, 9, 10
    ]
    
    for i, price in enumerate(gas_prices):
        X.loc[(X.date.dt.month == months[i]) & (X.date.dt.year == years[i]), 'gas_price'] = price

    return X

## Import main dataset

In [7]:
data = pd.read_parquet(path / 'train.parquet')
test = pd.read_parquet(path / 'final_test.parquet')

targets = ['bike_count', 'log_bike_count']

In [8]:
data.drop(columns=['site_name', 'counter_id', 'site_id', 'counter_installation_date', 'coordinates', 'counter_technical_id'], inplace=True)
test.drop(columns=['site_name', 'counter_id', 'site_id', 'counter_installation_date', 'coordinates', 'counter_technical_id'], inplace=True)

## Model

In [10]:
X, y = data.drop(columns=targets), data['log_bike_count']

In [11]:
X_train, y_train, X_test, y_test = train_test_split_temporal(X, y)

print(
    f'Train: n_samples={X_train.shape[0]},  {X_train["date"].min()} to {X_train["date"].max()}'
)
print(
    f'Test: n_samples={X_test.shape[0]},  {X_test["date"].min()} to {X_test["date"].max()}'
)

Train: n_samples=416187,  2020-09-01 01:00:00 to 2021-07-11 23:00:00
Test: n_samples=80640,  2021-07-12 00:00:00 to 2021-09-09 23:00:00


In [12]:
target_name = 'log_bike_count'

data_merger = FunctionTransformer(_merge_external_data, validate=False)
covid_encoder = FunctionTransformer(_encode_covid, validate=False)
gas_encoder = FunctionTransformer(_gas_price_encoder, validate=False)
date_encoder = FunctionTransformer(_encode_dates, validate=False)

date_cols = _encode_dates(X_train[["date"]]).columns.tolist()
categorical_cols = ["counter_name"] + date_cols

In [13]:
#{'learning_rate': 0.16263414906434522, 'n_estimators': 633}
best_params = {
    'learning_rate': 0.16,
    'max_depth': 8,
    'n_estimators': 630,
    'subsample': 0.8,
    'od_pval': 1e-5
}

regressor = CatBoostRegressor(**best_params)

pipe = Pipeline(
    [
        ('merge external', data_merger),
        ('gas prices encoder', gas_encoder),
        ('covid encoder', covid_encoder),
        ('date encoder', date_encoder),
        ('regressor',regressor)
    ]
)

In [14]:
val_pool = Pool(_encode_dates(_encode_covid(_gas_price_encoder(_merge_external_data(X_test)))), label=y_test, cat_features=categorical_cols)
pipe.fit(X_train, y_train, regressor__cat_features=categorical_cols, regressor__early_stopping_rounds=70, regressor__eval_set=val_pool)

0:	learn: 1.5014616	test: 1.2869925	best: 1.2869925 (0)	total: 465ms	remaining: 4m 52s
1:	learn: 1.3410312	test: 1.1445236	best: 1.1445236 (1)	total: 747ms	remaining: 3m 54s
2:	learn: 1.2128989	test: 1.0274292	best: 1.0274292 (2)	total: 961ms	remaining: 3m 20s
3:	learn: 1.1121300	test: 0.9490088	best: 0.9490088 (3)	total: 1.2s	remaining: 3m 7s
4:	learn: 1.0311879	test: 0.8812718	best: 0.8812718 (4)	total: 1.51s	remaining: 3m 8s
5:	learn: 0.9693008	test: 0.8341353	best: 0.8341353 (5)	total: 1.77s	remaining: 3m 4s
6:	learn: 0.8940034	test: 0.7854209	best: 0.7854209 (6)	total: 2.07s	remaining: 3m 4s
7:	learn: 0.8358639	test: 0.7545446	best: 0.7545446 (7)	total: 2.26s	remaining: 2m 55s
8:	learn: 0.7906048	test: 0.7386848	best: 0.7386848 (8)	total: 2.52s	remaining: 2m 54s
9:	learn: 0.7555866	test: 0.7234542	best: 0.7234542 (9)	total: 2.71s	remaining: 2m 48s
10:	learn: 0.7290885	test: 0.7190395	best: 0.7190395 (10)	total: 2.96s	remaining: 2m 46s
11:	learn: 0.7063947	test: 0.7104729	best: 0.7

94:	learn: 0.4872066	test: 0.6443326	best: 0.6439107 (89)	total: 31.3s	remaining: 2m 56s
95:	learn: 0.4862175	test: 0.6465744	best: 0.6439107 (89)	total: 31.8s	remaining: 2m 56s
96:	learn: 0.4855444	test: 0.6460930	best: 0.6439107 (89)	total: 32s	remaining: 2m 56s
97:	learn: 0.4843599	test: 0.6456770	best: 0.6439107 (89)	total: 32.5s	remaining: 2m 56s
98:	learn: 0.4834721	test: 0.6456296	best: 0.6439107 (89)	total: 32.8s	remaining: 2m 56s
99:	learn: 0.4825240	test: 0.6447389	best: 0.6439107 (89)	total: 33.1s	remaining: 2m 55s
100:	learn: 0.4821381	test: 0.6445323	best: 0.6439107 (89)	total: 33.5s	remaining: 2m 55s
101:	learn: 0.4815057	test: 0.6446755	best: 0.6439107 (89)	total: 33.8s	remaining: 2m 54s
102:	learn: 0.4804967	test: 0.6442864	best: 0.6439107 (89)	total: 34.2s	remaining: 2m 54s
103:	learn: 0.4798250	test: 0.6440488	best: 0.6439107 (89)	total: 34.5s	remaining: 2m 54s
104:	learn: 0.4792889	test: 0.6437360	best: 0.6437360 (104)	total: 34.8s	remaining: 2m 54s
105:	learn: 0.478

185:	learn: 0.4432726	test: 0.6294126	best: 0.6292869 (183)	total: 1m 3s	remaining: 2m 31s
186:	learn: 0.4427728	test: 0.6294097	best: 0.6292869 (183)	total: 1m 3s	remaining: 2m 31s
187:	learn: 0.4425984	test: 0.6289048	best: 0.6289048 (187)	total: 1m 4s	remaining: 2m 31s
188:	learn: 0.4421676	test: 0.6295509	best: 0.6289048 (187)	total: 1m 4s	remaining: 2m 30s
189:	learn: 0.4418388	test: 0.6292134	best: 0.6289048 (187)	total: 1m 4s	remaining: 2m 30s
190:	learn: 0.4416381	test: 0.6290146	best: 0.6289048 (187)	total: 1m 5s	remaining: 2m 29s
191:	learn: 0.4412795	test: 0.6285946	best: 0.6285946 (191)	total: 1m 5s	remaining: 2m 29s
192:	learn: 0.4410557	test: 0.6284472	best: 0.6284472 (192)	total: 1m 5s	remaining: 2m 29s
193:	learn: 0.4409462	test: 0.6282319	best: 0.6282319 (193)	total: 1m 6s	remaining: 2m 28s
194:	learn: 0.4403544	test: 0.6281250	best: 0.6281250 (194)	total: 1m 6s	remaining: 2m 28s
195:	learn: 0.4400883	test: 0.6281810	best: 0.6281250 (194)	total: 1m 6s	remaining: 2m 28s

275:	learn: 0.4213299	test: 0.6269692	best: 0.6260403 (270)	total: 1m 35s	remaining: 2m 2s
276:	learn: 0.4211943	test: 0.6269386	best: 0.6260403 (270)	total: 1m 35s	remaining: 2m 2s
277:	learn: 0.4210515	test: 0.6269883	best: 0.6260403 (270)	total: 1m 36s	remaining: 2m 1s
278:	learn: 0.4207751	test: 0.6268050	best: 0.6260403 (270)	total: 1m 36s	remaining: 2m 1s
279:	learn: 0.4206168	test: 0.6267583	best: 0.6260403 (270)	total: 1m 36s	remaining: 2m 1s
280:	learn: 0.4205210	test: 0.6267915	best: 0.6260403 (270)	total: 1m 37s	remaining: 2m
281:	learn: 0.4203882	test: 0.6268374	best: 0.6260403 (270)	total: 1m 37s	remaining: 2m
282:	learn: 0.4201649	test: 0.6275919	best: 0.6260403 (270)	total: 1m 37s	remaining: 2m
283:	learn: 0.4200027	test: 0.6276961	best: 0.6260403 (270)	total: 1m 38s	remaining: 1m 59s
284:	learn: 0.4198015	test: 0.6273111	best: 0.6260403 (270)	total: 1m 38s	remaining: 1m 59s
285:	learn: 0.4196634	test: 0.6272974	best: 0.6260403 (270)	total: 1m 38s	remaining: 1m 58s
286:	

365:	learn: 0.4071813	test: 0.6268925	best: 0.6255427 (297)	total: 2m 6s	remaining: 1m 30s
366:	learn: 0.4070810	test: 0.6266465	best: 0.6255427 (297)	total: 2m 6s	remaining: 1m 30s
367:	learn: 0.4069319	test: 0.6267726	best: 0.6255427 (297)	total: 2m 6s	remaining: 1m 30s
Stopped by overfitting detector  (70 iterations wait)

bestTest = 0.6255426711
bestIteration = 297

Shrink model to first 298 iterations.


Pipeline(steps=[('merge external',
                 FunctionTransformer(func=<function _merge_external_data at 0x000001F4BF013520>)),
                ('gas prices encoder',
                 FunctionTransformer(func=<function _gas_price_encoder at 0x000001F4DD13B760>)),
                ('covid encoder',
                 FunctionTransformer(func=<function _encode_covid at 0x000001F4DD175AB0>)),
                ('date encoder',
                 FunctionTransformer(func=<function _encode_dates at 0x000001F4DCF7B1C0>)),
                ('regressor',
                 <catboost.core.CatBoostRegressor object at 0x000001F4DD221C30>)])

In [15]:
y_hat = pipe.predict(X_test)

print(
    f"Train set, RMSE={mean_squared_error(y_train, pipe.predict(X_train), squared=False):.2f}"
)
print(
    f"Test set, RMSE={mean_squared_error(y_test, y_hat, squared=False):.2f}"
)

Train set, RMSE=0.40
Test set, RMSE=0.63


# Submission

In [ ]:
pipe.fit(X,y)
prediction = pipe.predict(test)
prediction[prediction<0] = 0

In [ ]:
submission = pd.DataFrame({'log_bike_count' : prediction})

# submission = pd.DataFrame({'Id' : submission.index, 'log_bike_count' : prediction})
submission = pd.DataFrame({'Id' : test.index, 'log_bike_count' : prediction})

submission.to_csv("submission.csv", index=False)